In [1]:
import plotly.express as px
import pandas as pd
import json
import os
os.environ['USER'] = "rmoine"
from pathlib import Path
import plotly.graph_objects as go
import numpy as np
from typing import *
from llama.main import compute_metrics_from_files, DatasetName, compute_metrics_from_list, get_tokenizer
import llama.main as m
from sklearn.metrics import f1_score, auc, roc_curve, confusion_matrix
from PIL import Image
from pandasql import sqldf
from io import BytesIO
import base64
from plotly.graph_objects import Figure
import win32clipboard
import textwrap
import colorsys
import optuna
import tqdm
import re
import random
def img_to_clipboard(fig: Figure):
    img_bytes = fig.to_image(format="png")
    image = Image.open(BytesIO(img_bytes))
    
    output = BytesIO()
    image.convert('RGB').save(output, 'BMP')
    data = output.getvalue()[14:]
    output.close()
    win32clipboard.OpenClipboard()
    win32clipboard.EmptyClipboard()
    win32clipboard.SetClipboardData(win32clipboard.CF_DIB, data)
    win32clipboard.CloseClipboard()
root = Path(f"../../data/baseline_studies/analysis/sel/")

c:\Users\robin\miniconda3\envs\severityPrediction\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
Import of import evaluate failed
Import of from pytorchtools import EarlyStopping failed


In [2]:
l = []
files = list(enumerate(root.rglob("*_0.json")))
for i,f in tqdm.tqdm(files):
    algorithm_group: Literal['bayesian_network','svc','knn']
    if "_nb_" in f.stem:
        algorithm_group = 'bayesian_network'
    elif "_svm_knn_" in f.stem:
        if "KNeighborsClassifier" in f.stem:
            algorithm_group = "knn"
        elif "SVC_" in f.stem:
            algorithm_group = "svc"
        else:
            raise Exception
    else:
        raise Exception
    num_samples = int(re.search("([0-9]+)_samples", f.stem).group(1))
            
    with open(f) as fp:
        data = json.load(fp)[0]
    dataset_choice = "eclipse_72k" if "eclipse_72k" in f.stem else "mozilla_200k"
    fixed_params = {k:v for k,v in data.items() if not isinstance(v, list)}
    fixed_params["params"] = fixed_params
    fixed_params["run_id"] = i
    fixed_params["num_samples"] = num_samples
    fixed_params["algorithm_group"] = algorithm_group
    fixed_params["dataset_choice"] = dataset_choice
    infos = {**fixed_params}
    for dataset in ["val","test"]:
        d = [
            {"binary_severity":b, "severity_pred": p, "bug_id": i} 
            for i,(b,p) in enumerate(zip(
                data[f'binary_severity_{dataset}'],
                data[f'y_{dataset}_pred']
                ))]
        threasholds = sorted(np.unique(filter(lambda x:x is not None,data[f'thresholds_{dataset}'])))
        fpr, tpr, thresholds = roc_curve(data[f'binary_severity_{dataset}'], data[f'y_{dataset}_pred'])
        auc_test = auc(fpr, tpr)
        fig = go.Figure()
        y_scores = data[f'y_{dataset}_pred']
        y_true = data[f'binary_severity_{dataset}']
        roc_data = []
        best_accuracy = 0
        best_threshold = -1
        best_data = None

        for i, threshold in enumerate(thresholds):
            y_pred = (y_scores >= threshold).astype(int)
            cm = confusion_matrix(y_true, y_pred)
            accuracy = np.sum(np.diag(cm))/np.sum(cm)*100
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_threshold = threshold
                best_data = y_true, y_pred
        assert best_data is not None
        d = [
            {"binary_severity":y_true, "severity_pred": y_pred, "bug_id": i} for i,(y_true, y_pred) in enumerate(zip(*best_data))]
        cm, f1, _ = compute_metrics_from_list(d, pred_field="severity_pred")
        accuracy = np.sum(np.diag(cm)) / np.sum(cm)
        print(f"{dataset=} {f.stem=} {auc_test=}")
        infos[f"thresholdtest_accuracy_{dataset}"] = accuracy
        infos[f"thresholdtest_f1_{dataset}"] = f1
        infos[f"threashold_used_{dataset}"] = best_threshold
    l.append(infos)
df = pd.DataFrame(l)

  0%|          | 0/36 [00:00<?, ?it/s]

dataset='val' f.stem='BernoulliNB_nb_bino_eclipse_72k_1000_samples_0' auc_test=0.7460453815334909
dataset='test' f.stem='BernoulliNB_nb_bino_eclipse_72k_1000_samples_0' auc_test=0.7473364753219431


  3%|▎         | 1/36 [00:26<15:23, 26.38s/it]

dataset='val' f.stem='BernoulliNB_nb_bino_eclipse_72k_2000_samples_0' auc_test=0.7403913248503835
dataset='test' f.stem='BernoulliNB_nb_bino_eclipse_72k_2000_samples_0' auc_test=0.7369830781956884


  6%|▌         | 2/36 [00:39<10:28, 18.49s/it]

...

 94%|█████████▍| 34/36 [53:51<04:34, 137.49s/it]

dataset='val' f.stem='SVC_svm_knn_mozilla_200k_500_samples_0' auc_test=0.7326326425866516
dataset='test' f.stem='SVC_svm_knn_mozilla_200k_500_samples_0' auc_test=0.7273838647113249


 97%|█████████▋| 35/36 [55:49<02:11, 131.82s/it]

dataset='val' f.stem='SVC_svm_knn_mozilla_200k_8000_samples_0' auc_test=0.8056577889065122
dataset='test' f.stem='SVC_svm_knn_mozilla_200k_8000_samples_0' auc_test=0.8045063818970057


100%|██████████| 36/36 [59:18<00:00, 98.85s/it] 


In [13]:
df.to_pickle("../../data/baseline_results_analysed.json")

In [15]:
df = pd.read_pickle("../../data/baseline_results_analysed.json")

In [19]:
dataset_choice = "eclipse_72k"
d = df.query(f"dataset_choice == '{dataset_choice}'")
d.sort_values("num_samples",inplace=True)
d = d.rename({"thresholdtest_accuracy_val": "Validation accuracy with best threshold", "thresholdtest_accuracy_test": "Test accuracy with best threshold"}, axis=1)
fig = px.line(
    d,
    x="num_samples",
    y="Validation accuracy with best threshold",
    color="algorithm_group",
    title=f"Accuracy reached for dataset {dataset_choice}",
    markers=True,
)
for i,trace in enumerate(fig.data):
    trace.name =  trace.name + ' validation'
fig2 = px.line(d, x="num_samples", y="Test accuracy with best threshold", color="algorithm_group")
for i,trace in enumerate(fig2.data):
    trace.name = trace.name + ' test'
fig2.update_traces(patch={"line": {"dash": 'dot'}}) 
for i in range(3):
    fig.add_trace(fig2.data[i])
fig.update_layout(
    font=dict(size=16)  # Set the default font size for all text elements in the figure
)
fig.update_yaxes(title_text="Accuracy with best threshold")
fig.show()

C:\Users\robin\AppData\Local\Temp\ipykernel_19428\246019619.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
dataset_choice = "mozilla_200k"
d = df.query(f"dataset_choice == '{dataset_choice}'")
d.sort_values("num_samples",inplace=True)
d = d.rename({"thresholdtest_accuracy_val": "Validation accuracy with best threshold", "thresholdtest_accuracy_test": "Test accuracy with best threshold"}, axis=1)
fig = px.line(
    d,
    x="num_samples",
    y="Validation accuracy with best threshold",
    color="algorithm_group",
    title=f"Accuracy reached for dataset {dataset_choice}",
    markers=True,
)
for i,trace in enumerate(fig.data):
    trace.name =  trace.name + ' validation'
fig2 = px.line(d, x="num_samples", y="Test accuracy with best threshold", color="algorithm_group")
for i,trace in enumerate(fig2.data):
    trace.name = trace.name + ' test'
fig2.update_traces(patch={"line": {"dash": 'dot'}}) 
for i in range(3):
    fig.add_trace(fig2.data[i])
fig.update_layout(
    font=dict(size=16)  # Set the default font size for all text elements in the figure
)
fig.update_yaxes(title_text="Accuracy with best threshold")
fig.show()

C:\Users\robin\AppData\Local\Temp\ipykernel_19428\2388561007.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [27]:
d = df.query("dataset_choice == 'eclipse_72k'").query("auc_val == auc_val.max()").query("thresholdtest_accuracy_val == thresholdtest_accuracy_val.max()")
display(d.iloc[0])
d = df.query("dataset_choice == 'mozilla_200k'").query("auc_val == auc_val.max()").query("thresholdtest_accuracy_val == thresholdtest_accuracy_val.max()")
display(d.iloc[0])

seed                                                                           0
classifier                                                                   SVC
train_fun                                                                  train
train_accuracy                                                          0.843209
fold_id                                                                        0
auc_val                                                                 0.756181
classifier_name                                                              SVC
alpha                                                                        NaN
fit_prior                                                                    NaN
binarize                                                                     NaN
auc_test                                                                0.756181
params                         {'seed': 0, 'classifier': 'SVC', 'train_fun': ...
run_id                      

seed                                                                           0
classifier                                                            GaussianNB
train_fun                                                                  train
train_accuracy                                                             0.786
fold_id                                                                        0
auc_val                                                                 0.710356
classifier_name                                                       GaussianNB
alpha                                                                        NaN
fit_prior                                                                    NaN
binarize                                                                     NaN
auc_test                                                                0.710356
params                         {'seed': 0, 'classifier': 'GaussianNB', 'train...
run_id                      